<a href="https://colab.research.google.com/github/jkinarthur/ai-text-detection-baseline/blob/main/ai_text_detection_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install + Imports**

In [2]:
!pip -q install pandas scikit-learn

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


### Create a small dataset (Human vs AI)

In [3]:
human_texts = [
    "I went to the market this morning and bought fresh tomatoes and bread.",
    "My daughter had homework today and we worked on it together after dinner.",
    "The weather is cold in Toronto, so I wore a jacket and gloves.",
    "I struggled with the assignment at first, but after practicing, it became easier.",
    "The meeting started late because of traffic, but we still finished the agenda.",
    "I enjoy singing in the choir because it brings peace and joy to my heart.",
    "Today I cooked rice and stew, and the whole family ate together.",
    "I was tired after work, so I took a short nap before continuing my tasks.",
    "I called my friend to check on him because he has not been feeling well.",
    "I read a few pages of a book before going to bed last night."
]

ai_texts = [
    "This comprehensive analysis highlights the multifaceted nature of the problem and proposes a robust solution framework.",
    "In conclusion, the results demonstrate significant improvements across key performance indicators under controlled settings.",
    "The system leverages advanced optimization strategies to enhance efficiency and stability across varying conditions.",
    "This approach provides a scalable methodology that aligns with modern computational requirements and best practices.",
    "The model integrates contextual representation learning with structured decision-making to improve downstream outcomes.",
    "Overall, the findings support the hypothesis that intelligent automation can increase productivity and reduce operational risks.",
    "The proposed architecture exhibits strong generalization and maintains consistent performance under diverse scenarios.",
    "This work contributes to the literature by introducing an efficient pipeline for real-world deployment.",
    "By leveraging statistical inference and predictive modeling, the solution offers improved reliability and interpretability.",
    "The framework demonstrates strong adaptability and can be extended to multiple domains with minimal adjustments."
]

texts = human_texts + ai_texts
labels = [0]*len(human_texts) + [1]*len(ai_texts)  # 0=Human, 1=AI

df = pd.DataFrame({"text": texts, "label": labels})
df.head()


,text,label
0,I went to the market this morning and bought f...,0
1,My daughter had homework today and we worked o...,0
2,"The weather is cold in Toronto, so I wore a ja...",0
3,"I struggled with the assignment at first, but ...",0
4,"The meeting started late because of traffic, b...",0


## Train/Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.3, random_state=42, stratify=df["label"]
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))


Train size: 14
Test size: 6


# Build Model Pipeline (TF-IDF + Logistic Regression)

In [5]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
preds = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds, target_names=["Human", "AI"]))


Accuracy: 0.8333333333333334

Confusion Matrix:
 [[3 0]
 [1 2]]

Classification Report:
               precision    recall  f1-score   support

       Human       0.75      1.00      0.86         3
          AI       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



# Test with your own sample text

In [6]:
samples = [
    "I went to church and enjoyed the service today.",
    "This framework provides a comprehensive and scalable approach for automated evaluation."
]

pred = model.predict(samples)
prob = model.predict_proba(samples)

for s, p, pr in zip(samples, pred, prob):
    label = "AI" if p==1 else "Human"
    print("Text:", s)
    print("Prediction:", label)
    print("Probabilities [Human, AI]:", pr)
    print("-"*60)


Text: I went to church and enjoyed the service today.
Prediction: Human
Probabilities [Human, AI]: [0.55912487 0.44087513]
------------------------------------------------------------
Text: This framework provides a comprehensive and scalable approach for automated evaluation.
Prediction: AI
Probabilities [Human, AI]: [0.43057612 0.56942388]
------------------------------------------------------------
